<a href="https://colab.research.google.com/github/afif-af/hanson_ml_scikitlearn_tensorflow/blob/main/chapter08handson_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np

X=np.array([[[1,2,3],
    [4,5,6]],
   [[7,8,9],
    [10,11,12]]])

In [20]:
X_centered=X-X.mean(axis=0)
U, s, Vt=np.linalg.svd(X_centered)
c1=Vt[0]
c2=Vt[1]

In [21]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reshaped = X.reshape(-1, X.shape[-1])
X2D = pca.fit_transform(X_reshaped)

In [22]:
pca.explained_variance_ratio_

array([1., 0.])

In [23]:
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784',as_frame=False)
X_train, y_train=mnist.data[:60_000],mnist.target[:60_000]
X_test, y_test=mnist.data[60_000:],mnist.target[60_000:]

pca=PCA()
pca.fit(X_train)
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum>=0.95)+1

In [24]:
pca=PCA(n_components=0.95)
X_reduced=pca.fit_transform(X_train)

In [25]:
pca.n_components_

np.int64(154)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

clf=make_pipeline(PCA(random_state=42),
                  RandomForestClassifier(random_state=42))
param_distrib={
    "pca__n_components":np.arange(10, 80),
    "randomforestclassifier__n_estimators":np.arange(50, 500)
}
rnd_search=RandomizedSearchCV(clf,param_distrib,n_iter=10, cv=3,random_state=42)
rnd_search.fit(X_train[:1000], y_train[:1000])

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'pca__n_components': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       6...
       414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478,
       479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
       492, 493, 494, 495, 496, 497, 498, 499])},
                   random_state=42)

In [27]:
print(rnd_search.best_params_)

{'randomforestclassifier__n_estimators': np.int64(475), 'pca__n_components': np.int64(57)}


In [28]:
X_recovered=pca.inverse_transform(X_reduced)

In [29]:
rnd_pca=PCA(n_components=154, random_state=42,svd_solver="randomized")
X_reduced=rnd_pca.fit_transform(X_train)



In [30]:
from sklearn.decomposition import IncrementalPCA
n_batches=100
inc_pca=IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
  inc_pca.partial_fit(X_batch)

X_reduced=inc_pca.transform(X_train)

In [31]:
filename="my_mnist.mmap"
X_mmap=np.memmap(filename,dtype='float32', mode='write', shape=X_train.shape)
X_mmap[:]=X_train
X_mmap.flush()

In [31]:
X_mmap=np.memmap(filename,dtype='float32',mode='readonly').reshape(-1,784)
batch_size=X_mmap.shape[0]
inc_pca=IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mmap)

In [32]:
from sklearn.random_projection import johnson_lindenstrauss_min_dim
m, e =5000, 0.1
d=johnson_lindenstrauss_min_dim(m, eps=e)
d

np.int64(7300)

In [33]:
n=20000
np.random.seed(42)
P=np.random.randn(d,n)/np.sqrt(d)

X=np.random.randn(m,n)
X_reduced=X @ P.T

In [34]:
from sklearn.random_projection import GaussianRandomProjection

gaussian_rnd_proj=GaussianRandomProjection(eps=e, random_state=42)
X_reduced=gaussian_rnd_proj.fit_transform(X)

In [35]:
components_pinv = np.linalg.pinv(gaussian_rnd_proj.components_)
X_recovered = X_reduced @ components_pinv.T

In [36]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_unrolled = lle.fit_transform(X_swiss)